In [1]:
import pandas as pd
import copy
import numpy as np
import math

In [2]:
!tar -xf /content/final_project.tar.gz

1. Read the gene file (gene_annotation.txt) to get the list of unique genes.

2. Read the read length file (readsLen.txt) to store the length value of each gene

3. Read each line in the mapping file (tophat_read_mapping_on_genome.summary) to get the aligned location of each read on the chromosome. The aligned location of the read should plus the read's length to derive the aligned region (start_location to end_location) on the chromosome. Then loop over all genes in the file (gene_annotation.txt) to find the gene whose exon overlaps this aligned region. Once found, add one count to the gene id. Continue the search to the next line in file (tophat_read_mapping_on_genome.summary). 

Important criteria: 

a. A read is defined to overlap an exon if at least one read base is found to overlap the exon region (either 5' or 3'). 

b. If a read overlaps multiple exons of the same gene, the read should be counted exactly once for this gene. 

c. If a read overlaps multiple exons of different genes (this happens when two or more genes overlap in the genome),  this read should be discarded and never counted to any gene. 


**Task 1.**

Read the gene file (gene_annotation.txt) to get the list of unique genes.

In [3]:
# Read gene annotation file to dataframe
genes = pd.read_csv("/content/final_project/gene_annotation.txt", sep = '\t')
# ChromosomeID column is a mix of numbers and strings, typecast to string for easy processing.
genes['ChromosomeID'] = genes['ChromosomeID'].astype(str)
# Dataframe check
genes

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,GeneID,ChromosomeID,Exon,Loc_Start,Loc_End
0,Zm00001d027230,1,exon,44289,44947
1,Zm00001d027230,1,exon,45666,45803
2,Zm00001d027230,1,exon,45888,46133
3,Zm00001d027230,1,exon,46229,46342
4,Zm00001d027230,1,exon,46451,46633
...,...,...,...,...,...
402554,Zm00001d000232,B73V4_ctg98,exon,67064,67355
402555,Zm00001d000232,B73V4_ctg98,exon,67390,67515
402556,Zm00001d000232,B73V4_ctg98,exon,67605,68060
402557,Zm00001d000233,B73V4_ctg98,exon,96278,98668


**Task 2.**

Read the read length file (readsLen.txt) to store the length value of each gene

In [4]:
# Import read length file to dataframe
lengths = pd.read_csv("/content/final_project/readsLen.txt", sep = '\t')
# Get a list of read names from dataframe
read_names = lengths.Read_name.values.tolist()
# Get rid of whitespace in read name list
read_names = [s.strip() for s in read_names]
# Get a list of read lengths from list
read_len = lengths.Length.values.tolist()
# Dataframe check
lengths

,Read_name,Length
0,NB501332:109:HM5KTBGX2:1:11101:8851:1140,19
1,NB501332:109:HM5KTBGX2:1:11101:16091:1155,22
2,NB501332:109:HM5KTBGX2:1:11101:17993:1190,19
3,NB501332:109:HM5KTBGX2:1:11101:20861:1202,24
4,NB501332:109:HM5KTBGX2:1:11101:24939:1307,21
...,...,...
282863,NB501332:109:HM5KTBGX2:4:23612:11771:20073,22
282864,NB501332:109:HM5KTBGX2:4:23612:20607:20241,19
282865,NB501332:109:HM5KTBGX2:4:23612:12751:20268,21
282866,NB501332:109:HM5KTBGX2:4:23612:11531:20270,24


**Task 3.**
Read each line in the mapping file (tophat_read_mapping_on_genome.summary) to get the aligned location of each read on the chromosome. The aligned location of the read should plus the read's length to derive the aligned region (start_location to end_location) on the chromosome. Then loop over all genes in the file (gene_annotation.txt) to find the gene whose exon overlaps this aligned region. Once found, add one count to the gene id. Continue the search to the next line in file (tophat_read_mapping_on_genome.summary).

In [5]:
# Read mapping file into data frame
mapping_lines = pd.read_csv("/content/final_project/tophat_read_mapping_on_genome.summary", sep = '\t')
# Check what chromosomes we have mapping data for
print(mapping_lines.ChromosomeID.unique().tolist())
# Dataframe check
mapping_lines

['1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'Mt', 'Pt']


,Read_Name,ChromosomeID,Start_position
0,NB501332:109:HM5KTBGX2:2:13101:10049:3550,1,23095
1,NB501332:109:HM5KTBGX2:1:22104:15315:16290,1,43372
2,NB501332:109:HM5KTBGX2:2:22207:8081:13346,1,48293
3,NB501332:109:HM5KTBGX2:1:21306:12413:11166,1,50006
4,NB501332:109:HM5KTBGX2:2:12208:9550:8286,1,56865
...,...,...,...
101181,NB501332:109:HM5KTBGX2:2:21104:2602:19376,Pt,138642
101182,NB501332:109:HM5KTBGX2:4:13408:19501:17142,Pt,138642
101183,NB501332:109:HM5KTBGX2:2:23212:24588:2225,Pt,138681
101184,NB501332:109:HM5KTBGX2:1:23207:1810:7168,Pt,140141


In [6]:
# Filter gene annotation so only chromosome from the read mapping file are considered. This reduces the size of the df to be parsed by algorithm.
#genes = genes[genes['ChromosomeID'].isin(mapping_lines.ChromosomeID.unique().tolist())]
# Check unique chromosomes
print(genes.ChromosomeID.unique().tolist())
# Check gene annotation dataframe
genes

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Mt', 'Pt', 'B73V4_ctg10', 'B73V4_ctg100', 'B73V4_ctg102', 'B73V4_ctg103', 'B73V4_ctg104', 'B73V4_ctg105', 'B73V4_ctg107', 'B73V4_ctg11', 'B73V4_ctg110', 'B73V4_ctg115', 'B73V4_ctg117', 'B73V4_ctg119', 'B73V4_ctg12', 'B73V4_ctg120', 'B73V4_ctg121', 'B73V4_ctg123', 'B73V4_ctg125', 'B73V4_ctg129', 'B73V4_ctg13', 'B73V4_ctg133', 'B73V4_ctg134', 'B73V4_ctg14', 'B73V4_ctg140', 'B73V4_ctg141', 'B73V4_ctg142', 'B73V4_ctg144', 'B73V4_ctg147', 'B73V4_ctg150', 'B73V4_ctg151', 'B73V4_ctg170', 'B73V4_ctg173', 'B73V4_ctg178', 'B73V4_ctg18', 'B73V4_ctg180', 'B73V4_ctg181', 'B73V4_ctg182', 'B73V4_ctg188', 'B73V4_ctg189', 'B73V4_ctg190', 'B73V4_ctg193', 'B73V4_ctg2', 'B73V4_ctg20', 'B73V4_ctg205', 'B73V4_ctg206', 'B73V4_ctg208', 'B73V4_ctg219', 'B73V4_ctg23', 'B73V4_ctg24', 'B73V4_ctg242', 'B73V4_ctg245', 'B73V4_ctg247', 'B73V4_ctg248', 'B73V4_ctg250', 'B73V4_ctg253', 'B73V4_ctg26', 'B73V4_ctg28', 'B73V4_ctg29', 'B73V4_ctg3', 'B73V4_ctg31', 'B73V4_ct

,GeneID,ChromosomeID,Exon,Loc_Start,Loc_End
0,Zm00001d027230,1,exon,44289,44947
1,Zm00001d027230,1,exon,45666,45803
2,Zm00001d027230,1,exon,45888,46133
3,Zm00001d027230,1,exon,46229,46342
4,Zm00001d027230,1,exon,46451,46633
...,...,...,...,...,...
402554,Zm00001d000232,B73V4_ctg98,exon,67064,67355
402555,Zm00001d000232,B73V4_ctg98,exon,67390,67515
402556,Zm00001d000232,B73V4_ctg98,exon,67605,68060
402557,Zm00001d000233,B73V4_ctg98,exon,96278,98668


In [7]:
# Modified binary search algorithm to find index of first exon with an ending location >= to the read start.
def endSearch(arr, x, low, high):
  # Check for binary convergence
  if low == high:
    # If chosen index fufills (exon end >= read start) return index
    if x <= arr[low]:
      return low
    # Else return failure state.
    return -1
  # Not yet converged requires recursive call.
  else:
    # Find midpoint
    mid = math.floor((low + high) / 2)
    # If perfect match return index
    if x == arr[mid]:
      return mid
    # If read start is greater than gene end value at midpoint, discard lesser half.
    elif (x > arr[mid]):
      return endSearch(arr, x, mid+1, high)
    # If read start is less than exon end value at midpoint, discard greater half but keep midpoint.
    else: 
      return endSearch(arr, x, low, mid)

#endSearch(list(loc_end_list, gene_start, 0, 10)
print(list(range(10)))
print(endSearch(list(range(10)), 4, 0, 10)) 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
4


In [8]:
# Modified binary search algorithm to find index of first exon with an starting location <= to the read end.
def startSearch(arr, x, low, high):
  # Check for binary convergence
  if low == high:
    # If chosen index fufills (exon start =< read end) return index
    if x >= arr[high]:
      return low
    # Else return failure state.
    return -1
  # Not yet converged requires recursive call.
  else:
    # Find midpoint
    mid = math.ceil((low + high) / 2)
    # If perfect match return index
    if x == arr[mid]:
      return mid
    elif (x > arr[mid]):
    # If read end is greater than exon start value at midpoint, discard lesser half but keep midpoint.
      return startSearch(arr, x, mid, high)
    # If read end is less than exon start value at midpoint, discard greater half.
    else: 
      return startSearch(arr, x, low, mid-1)

#startSearch(list(loc_start_list, gene_end, 0, 10)
print(list(range(10)))
print(startSearch(list(range(10)), 3, 0, 10)) 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
3


O(n) efficiency gene count algorithm.

In [9]:
%%time
# Deep copy to preserve original data
work = copy.deepcopy(genes)
# Converting read mapping dataframe to dictionary because iterating through pandas df is non-efficient.
read_dict = mapping_lines.to_dict('records')
# List of genes to be counted, each occurence of gene in this list counts as +1.
counted_gene = []

# Index based dict parsing is most efficient.
for i in range(len(read_dict)):
    if (i%5000 == 0) : print('Iteration', i)

    # Temp data structure holds individual row
    temp = read_dict[i]

    # Pulls read name from temp.
    read_name = temp['Read_Name']
    # Find index of read name in read names list.
    index = read_names.index(read_name) # O(n)
    # Index matches read length order, pulls length
    length = read_len[index]
    # Pull chromosome string
    chrom = temp['ChromosomeID']
    # Pull read start position
    start = temp['Start_position']
    # Calculate read end position
    end = start + length  

    # Numpy filter gene dataframe by chromosome, gets index of all data with chromosome == chrom
    x = np.where((work['ChromosomeID']==chrom)) # O(n)
    # Filter np.array by index
    new = work.loc[x]
    # Pull exon end positions from np.array
    array = new.iloc[:,4].values
    # Get first index where exon end position >= read start position
    index_start = endSearch(array, start, 0, len(array)-1) # O(logn)
    # Pull exon start positions from np.array
    array = new.iloc[:,3].values
    # Get last index where exon start position <= read end
    index_end = startSearch(array, end, 0, len(array)-1) # O(logn)

    # If indices are not equal or start > end then there is no overlap
    if index_start > index_end: continue
    # Get uniuqe list of genes that overlap
    gene_list = set(new.reset_index().loc[index_start:index_end].GeneID)
    # If only 1 gene overlaps it is counter, multiple or no overlaps are not counted
    if (len(gene_list) == 1):
      counted_gene.append(str(gene_list))

Iteration 0
Iteration 5000
Iteration 10000
Iteration 15000
Iteration 20000
Iteration 25000
Iteration 30000
Iteration 35000
Iteration 40000
Iteration 45000
Iteration 50000
Iteration 55000
Iteration 60000
Iteration 65000
Iteration 70000
Iteration 75000
Iteration 80000
Iteration 85000
Iteration 90000
Iteration 95000
Iteration 100000
CPU times: user 1h 22s, sys: 19.1 s, total: 1h 41s
Wall time: 1h 40s


This algorithm follows the same logic as above, but uses np.where instead of our custom binary search. This checks to ensure that the binary searches run correctly.

In [10]:
%%time
work = copy.deepcopy(genes)
read_dict = mapping_lines.to_dict('records')
counted_gene_alt = []

for i in range(len(read_dict)):
    if (i%5000 == 0) : print('Iteration', i)
    temp = read_dict[i]

    read_name = temp['Read_Name']
    index = read_names.index(read_name)
    length = read_len[index]
    chrom = temp['ChromosomeID']
    start = temp['Start_position']
    end = start + length  

    x = np.where((work['ChromosomeID']==chrom))
    new = work.loc[x]
    y = np.where((new['Loc_End']>=start) & (new['Loc_Start']<= end))

    gene_list = set(new.reset_index().loc[y].GeneID)
    if (len(gene_list) == 1):
      counted_gene_alt.append(str(gene_list))

Iteration 0
Iteration 5000
Iteration 10000
Iteration 15000
Iteration 20000
Iteration 25000
Iteration 30000
Iteration 35000
Iteration 40000
Iteration 45000
Iteration 50000
Iteration 55000
Iteration 60000
Iteration 65000
Iteration 70000
Iteration 75000
Iteration 80000
Iteration 85000
Iteration 90000
Iteration 95000
Iteration 100000
CPU times: user 1h 5min 45s, sys: 24.5 s, total: 1h 6min 10s
Wall time: 1h 6min 9s


This writes output to file so we don't have to rerun long algorithms.

In [12]:
with open('binary.txt', 'w+') as f:
    for items in counted_gene:
        f.write('%s\n' %items)
f.close()

In [13]:
with open('numpy.txt', 'w+') as f:
    for items in counted_gene_alt:
        f.write('%s\n' %items)
f.close()

In [14]:
# Get list of all genes to be counted from the gene dataframe
genes_list = genes.GeneID.tolist()
# Get a copy of the list of all genes that aligned to a read at a ratio of 1 gene : 1 read.
work_list = copy.deepcopy(counted_gene)
# Each gene that aligned to a read gets counted once
count_list = [1] * len(work_list)
# The mast list of genes are not counted
nocount_list = [0] * len(genes_list)
# Append the list of counted genes with non counted genes and merge the count lists
work_list.extend(genes_list)
count_list.extend(nocount_list)
# Form a dataframe of genes and counts
d = {'Genes':work_list,'Count':count_list}
df = pd.DataFrame(d)
df = df.reset_index(inplace=False)
df['Genes'] = df['Genes'].str.replace(r"'}", '', regex=True)
df['Genes'] = df['Genes'].str.replace(r"{'", '', regex=True)
# Group genes by name and sum counts so each gene is unique with a single count value
df = df.groupby('Genes').sum()
# Save data to dataframe
df.to_csv("data.csv", index=True)
# View data
df

,index,Count
Genes,,
GRMZM5G800096,2381428,32
GRMZM5G800101,979945473,23952
GRMZM5G800457,459653,0
GRMZM5G800780,1358429,15
GRMZM5G800980,3518532,56
...,...,...
zma-MIR482,87617,0
zma-MIR528a,63043,0
zma-MIR528b,773162,12


Note we returned the same number of genes as the tool calculation at: 41942.

Our count matched the tool for of 38790/41942 genes, an accuracy of 92.5%.



In [15]:
# Read tool calculated values to compare
control = pd.read_csv("/content/final_project/countfile.txt", sep = '\t', header = None)
# Add our custom counts to the dataframe
control['Calculated'] = df.Count.values.tolist()
# Find the difference between our gene counts and theirs
control['Score_diff'] = control['Calculated'] - control[1]
# Filter the dataframe so only genes that differ in count are showing
control = control[control['Score_diff'] != 0]
control

,0,1,Calculated,Score_diff
5,GRMZM5G801074,28,0,-28
10,GRMZM5G804708,74,45,-29
22,GRMZM5G813608,179,160,-19
27,GRMZM5G815606,8,12,4
41,GRMZM5G825253,1,0,-1
...,...,...,...,...
41909,zma-MIR395l,0,1,1
41919,zma-MIR396f,0,3,3
41925,zma-MIR398b,135,139,4
41936,zma-MIR408b,190,191,1


To see where we disagreed I pulled a gene with differing counts.

Notice how our algorithm does not include it but the tool count does.

In [16]:
x = np.where(genes['GeneID'] == 'GRMZM5G801074')
new = genes.loc[x]
new

,GeneID,ChromosomeID,Exon,Loc_Start,Loc_End
399580,GRMZM5G801074,Pt,exon,66998,67198


When we check if any reads in the mapping lines fufills this gene, we get no results. This supports the accuracy of our algorithm over the tool. I believe a difference in parameters or chromosome read regions causes this. Notice our reads did not cover all the chromosomes included in the gene file.

In [17]:
y = np.where((mapping_lines['Start_position']>=66998) & mapping_lines['ChromosomeID'] == 'Pt')
mapping_lines.loc[y]

,Read_Name,ChromosomeID,Start_position


**EVERYTHING BELOW THIS IS EFFICIENCY TESTING**

In [18]:
%%timeit
array = new.iloc[:,4].values
index_start = endSearch(array, start, 0, len(array)-1)
array = new.iloc[:,3].values
index_end = startSearch(array, end, 0, len(array)-1)
gene_list = set(new.reset_index().loc[index_start:index_end].GeneID)
gene_list

796 µs ± 10 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
array = new.iloc[:,3].values
startSearch(array, end, 0, len(array)-1)

0

In [20]:
%%timeit
y = np.where((new['Loc_End']>=start) & (new['Loc_Start']<= end))
gene_list = set(new.reset_index().loc[y].GeneID)
gene_list

1.38 ms ± 26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
array = new.iloc[:,4].values
endSearch(array, start, 0, len(array)-1)

-1

In [22]:
%%time
work = copy.deepcopy(genes)
df_searchable = work.set_index('ChromosomeID')
count_dict = {}
read_dict = mapping_lines.to_dict('records')

for i in range(len(read_dict)):
    if (i == 50): break
    #if (i%5000 == 0) : print('Iteration', i)
    temp = read_dict[i]

    read_name = temp['Read_Name']
    index = read_names.index(read_name)
    length = read_len[index]
    chrom = temp['ChromosomeID']
    start = temp['Start_position']
    end = start + length
    

    filtered_work = work[work['ChromosomeID'] == chrom] 
    filtered_work = filtered_work[filtered_work['Loc_End'] > start]
    filtered_work = filtered_work[filtered_work['Loc_Start'] < end]
    gene_list = filtered_work.GeneID.unique().tolist()
    print(gene_list)

    if (len(gene_list) == 1):
      gene_id = gene_list[0]
      if (gene_id in count_dict):
        count_dict[gene_id] += 1
      else:
        count_dict.setdefault(gene_id, 1)

[]
[]
['Zm00001d027230']
[]
[]
[]
[]
[]
[]
['Zm00001d027234']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Zm00001d027254']
[]
[]
[]
[]
[]
['Zm00001d027266']
['Zm00001d027266']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Zm00001d027276']
[]
[]
[]
[]
[]
[]
CPU times: user 2.5 s, sys: 45.1 ms, total: 2.55 s
Wall time: 2.55 s


In [23]:
%%timeit
filtered_work = df_searchable[df_searchable.index == chrom]
filtered_work = filtered_work[filtered_work['Loc_End'] > start]
filtered_work = filtered_work[filtered_work['Loc_Start'] < end]
filtered_work

37.5 ms ± 602 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%%timeit
x = np.where((work['ChromosomeID']==chrom))
new = work.loc[x]
y = np.where((new['Loc_End']>=start) & (new['Loc_Start']< end))
new.loc[y]

38.6 ms ± 770 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit
gene_list = new.loc[y].GeneID.unique().tolist()
if (len(gene_list) == 1):
  counted_gene.append(gene_list[0])

401 µs ± 33.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
%%timeit
gene_list = set(new.loc[y].GeneID)
if (len(gene_list) == 1):
  counted_gene.append(str(gene_list))

316 µs ± 28.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%%timeit
x = np.where((work['Loc_End']>=start) & (work['Loc_Start']< end) & (work['ChromosomeID']==chrom))
work.loc[x]

32 ms ± 801 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%%timeit
filtered_work = df_searchable[df_searchable.index == chrom]
x = np.where((filtered_work['Loc_End']>=start) & (filtered_work['Loc_Start']< end))
filtered_work.loc[x]

36.6 ms ± 747 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%%timeit
filtered_work = work[work['ChromosomeID'] == chrom] 
filtered_work = filtered_work[filtered_work['Loc_End'] > start]
filtered_work = filtered_work[filtered_work['Loc_Start'] < end]
gene_list = filtered_work.GeneID.unique().tolist()

38.3 ms ± 566 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit
filtered_work = df_searchable[df_searchable.index == chrom]
filtered_work = filtered_work[filtered_work['Loc_End'] > start]
filtered_work = filtered_work[filtered_work['Loc_Start'] < end]
gene_list = filtered_work.GeneID.unique().tolist()

37.9 ms ± 726 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%%time
work = copy.deepcopy(genes)
df_searchable = work.set_index('ChromosomeID')
count_dict = {}
read_dict = mapping_lines.to_dict('records')

genes_list = genes.GeneID.tolist()
chrom_list = genes.ChromosomeID.tolist()
start_list = genes.Loc_Start.tolist()
end_list = genes.Loc_End.tolist()

counted_gene = []

for i in range(len(read_dict)):
    if (i%10000 == 0) : print('Iteration', i)
    temp = read_dict[i]
    break

    read_name = temp['Read_Name']
    index = read_names.index(read_name)
    length = read_len[index]
    chrom = temp['ChromosomeID']
    start = temp['Start_position']
    end = start + length

    chrom_index = [a for a,v in enumerate(chrom_list) if v == chrom]
    start_index = [a for a,v in enumerate(end_list) if v > start]    
    end_index = [a for a,v in enumerate(start_list) if v < end]
    matches = set(start_index) & set(end_index) & set(chrom_index)
    res_list = set([genes_list[b] for b in matches])

    if (len(res_list) == 1):
      counted_gene.append([res_list][0])

Iteration 0
CPU times: user 375 ms, sys: 54.1 ms, total: 429 ms
Wall time: 429 ms


In [32]:
%%timeit
filtered_work = work[work['ChromosomeID'] == chrom] 
filtered_work = filtered_work[filtered_work['Loc_End'] > start]
filtered_work = filtered_work[filtered_work['Loc_Start'] < end]
filtered_work

38 ms ± 829 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
filtered_work = df_searchable[df_searchable.index == chrom]
filtered_work = filtered_work[filtered_work['Loc_End'] > start]
filtered_work = filtered_work[filtered_work['Loc_Start'] < end]
filtered_work

37.4 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%%time
work = copy.deepcopy(genes)
df_searchable = work.set_index('ChromosomeID')
count_dict = {}
read_dict = mapping_lines.to_dict('records')
counted_gene = []

for i in range(len(read_dict)):
    if (i%5000 == 0) : print('Iteration', i)
    temp = read_dict[i]

    read_name = temp['Read_Name']
    index = read_names.index(read_name)
    length = read_len[index]
    chrom = temp['ChromosomeID']
    start = temp['Start_position']
    end = start + length
    

    filtered_work = df_searchable[df_searchable.index == chrom]
    filtered_work = filtered_work[filtered_work['Loc_End'] > start]
    filtered_work = filtered_work[filtered_work['Loc_Start'] < end]
    gene_list = filtered_work.GeneID.unique().tolist()

    if (len(gene_list) == 1):
      break
      counted_gene.append(gene_list[0])

Iteration 0
CPU times: user 494 ms, sys: 43 ms, total: 537 ms
Wall time: 537 ms
